In [6]:
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np
import requests
import re

In [ ]:
from IPython.display import JSON # this is ftom chris to see our json and navigate the tree 
display(JSON(r))

In [7]:
city_names = ['Berlin', 'Vienna', 'Rome', 'Paris', 'Barcelona', 'Milan', 'Amsterdam', 'Stockholm', 'Oslo', 'Helsinki', 'Prague',
          'Madrid', 'Budapest', 'Madrid', 'Copenhagen', 'Athens', 'London', 'Cologne', 'Hamburg', 'Munich', 'Bucharest', 
          'Dublin', 'Edinburgh', 'Sofia']

country = []
coordinates = []
population = []

def extract_population(text):
    numbers = re.findall(r'\b\d{1,3}(?:,\d{3})+\b', text)
    return max(map(lambda num: int(num.replace(',', '')), numbers))

for city in city_names:
    url = f"https://en.wikipedia.org/wiki/{city}"
    
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, "html.parser")
        # selecting country
        for s in soup.select("table.infobox tbody tr th"):
            if s.text == "Country":
                try:
                    country.append(s.find_next_sibling("td").select("a")[0].get_text())
                except:
                    country.append(s.find_next_sibling("td").get_text())
                break
        # selecting coordinates
        for s in soup.select("table.infobox tbody tr td"):
            if s.text.startswith("Coordinates: "):
                coordinates.append(s.text.split("/")[-1].split(";"))
        # selecting population
        population.append(extract_population(soup.find('table', class_='infobox').text))

cities_df = pd.DataFrame(
    {"City" : city_names,
    "Country" : country,
    "Coordinates" : coordinates,
    "Population" : population
    }
)


In [8]:
cities_df

,City,Country,Coordinates,Population
0,Berlin,Germany,"[ 52.52000, 13.40500]",6144600
1,Vienna,Austria,"[ 48.20833, 16.37250]",951354
2,Rome,Italy,"[ 41.89333, 12.48278]",860009
3,Paris,France,"[ 48.85667, 2.35222]",13024518
4,Barcelona,Spain,"[ 41.38278, 2.17694]",840000
5,Milan,Italy,"[ 45.46694, 9.19000]",371498
6,Amsterdam,Netherlands,"[ 52.37278, 4.89361]",480394
7,Stockholm,Sweden,"[ 59.32944, 18.06861]",2121000
8,Oslo,Norway,"[ 59.91333, 10.73889]",64235
9,Helsinki,Finland,"[ 60.17083, 24.93750]",559558
